In [ ]:
# Define roll number parameters
r1, r2, r3 = 3, 2, 3
theta = r1 / 10 + r2 / 100 + r3 / 1000

In [ ]:
# prompt: ### **Problem Statement**
# 1. Define a unitary operator $U$ based on the last three digits of your roll number $ r_1, r_2, r_3 $
# For my roll number `0424062323`:
#    - $ r_1 = 3 $
#    - $ r_2 = 2 $
#    - $ r_3 = 3 $
#    - Therefore, $ \theta = \frac{3}{10} + \frac{2}{100} + \frac{3}{1000} = 0.323 $ radians.
#    - Let $ \theta = \frac{r_1}{10} + \frac{r_2}{100} + \frac{r_3}{1000} $ radians.
#    - Define a unitary operator $ U = R_z(\theta) $ (rotation around the Z-axis by $ \theta $ radians).

import numpy as np

# Define the unitary operator U = Rz(theta)
def rz(theta):
  """
  Returns the matrix representation of the rotation around the Z-axis by theta radians.
  """
  return np.array([
      [np.exp(-1j * theta / 2), 0],
      [0, np.exp(1j * theta / 2)]
  ])

U = rz(theta)

print("Unitary operator U:")
U


In [ ]:
! pip install qiskit
! pip install qiskit-ibm-runtime
! pip install qiskit[visualization]
! pip install qiskit_aer

In [ ]:
# from qiskit import QuantumCircuit, Aer, execute

# from qiskit import QuantumCircuit, Aer, execute

# Modified import:
from qiskit import QuantumCircuit
from qiskit_aer import Aer  # Import Aer from qiskit_aer
from qiskit.visualization import plot_histogram
from qiskit.primitives import Sampler # Import Sampler instead of execute

# ... rest of the code ...
# ... rest of the code ...

# Define roll number parameters
r1, r2, r3 = 3, 2, 3
theta = r1 / 10 + r2 / 100 + r3 / 1000

# Define the unitary operator U = Rz(theta)
def rz(theta):
  """
  Returns the matrix representation of the rotation around the Z-axis by theta radians.
  """
  return np.array([
      [np.exp(-1j * theta / 2), 0],
      [0, np.exp(1j * theta / 2)]
  ])

U = rz(theta)

# Quantum Phase Estimation
n_count = 3  # Number of qubits for the eigenvalue register
n_state = 1  # Number of qubits for the state register

qc = QuantumCircuit(n_count + n_state, n_count)

# Initialize state register to |1>
qc.x(3)

# Apply Hadamard gates to the counting qubits
for i in range(n_count):
    qc.h(i)

# Apply controlled-U gates
for i in range(n_count):
    qc.cp(2 * np.pi / (2**(i+1)), i, 3)  # Note the 2*pi factor here

# Apply inverse Quantum Fourier Transform (QFT)
for i in range(n_count // 2):
    qc.swap(i, n_count - i - 1)
for i in range(n_count):
    for j in range(i):
        qc.cp(-np.pi / (2**(i - j)), j, i)
    qc.h(i)

# Measure the counting qubits
for i in range(n_count):
    qc.measure(i, i)

# Simulate the circuit
simulator = Aer.get_backend('qasm_simulator')

# Use Sampler instead of execute
sampler = Sampler()
job = sampler.run(qc, shots=1024)
result = job.result()
counts = result.quasi_dists[0].binary_probabilities() # get counts from quasi_dists


# Print the results
print("Estimated Phase (counts):", counts)

# Convert counts to phase
estimated_phase = max(counts, key=counts.get)
estimated_phase_decimal = int(estimated_phase, 2) / (2**n_count)

print("Estimated Phase (decimal):", estimated_phase_decimal)
print("Actual theta:", theta)

# Plot the histogram
plot_histogram(counts)